# **Import required packages**

In [ ]:
!pip install yfinance sqlite-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 1.9 MB/s eta 0:00:00


# **Mount Google Drive folder for file access**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **Load yfinance data into SQLite3**

1. Mount Google Drive and create an SQLite database file.
2. Define the table structure for storing financial data.
3. Helper functions for data extraction and conversion
4. Fetch 10-K reports for NASDAQ-100 companies using `yfinance` and store them in the database.



In [ ]:
import sqlite3
import yfinance as yf
import pandas as pd
import json
import re

db_path = "/content/drive/My Drive/FinScope3D/Structured_Data/nasdaq100.db"

**1. Create an SQLite database file.**

**2. Define the table structure for storing financial data.**

In [ ]:
# Connect to (or create) a SQLite database file inside Google Drive
# This ensures data persistence between Colab sessions
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create a table named 'financials' if it doesn't already exist
# This table stores financial data for each ticker by year, type, and metric
cursor.execute("""
    CREATE TABLE IF NOT EXISTS financials (
        ticker TEXT,
        year INTEGER,
        data_type TEXT,
        metric_name TEXT,
        value REAL,
        PRIMARY KEY (ticker, year, data_type, metric_name)
    )
""")

# Save changes to the database
conn.commit()
conn.close()

print(f"✅ Database saved to Google Drive: {db_path}")

✅ 資料庫已儲存到 Google Drive: /content/drive/My Drive/Colab Notebooks/structured_data/nasdaq100.db


In [ ]:
# Reconnect to the database to confirm the table exists
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())  # If result is [], table creation failed

[('financials',)]


**3. Helper functions for data extraction and conversion**

In [ ]:
# Function to extract year from a date string (e.g., "2023-09-30" → 2023)
def extract_year(date_str):
    match = re.search(r"\d{4}", date_str)
    return int(match.group()) if match else None

In [ ]:
# Save parsed JSON financial data into SQLite
def save_financials_to_db(ticker, data_type, data):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    data_dict = json.loads(data)  # Convert JSON string to Python dict

    for year_key, metrics in data_dict.items():
        year = extract_year(year_key)
        if not year:
            print(f"❌ Invalid date format for {ticker}: {year_key}")
            continue

        for metric, value in metrics.items():
            if value is None:
                continue  # Skip missing values

            cursor.execute("""
                INSERT OR REPLACE INTO financials (ticker, year, data_type, metric_name, value)
                VALUES (?, ?, ?, ?, ?)
            """, (ticker, year, data_type, metric, value))

    conn.commit()
    conn.close()

In [ ]:
# Convert a DataFrame's columns into years and transpose it
def convert_columns_to_year(df):
    if df is None or df.empty:
        return "{}"  # Return empty JSON if DataFrame is empty

    df = df.T  # Transpose: columns → rows, rows → columns
    df.index = pd.to_datetime(df.index, errors="coerce").year.astype(str)  # Set index as year strings

    return df.to_json(orient="index")  # Convert to JSON (dict by year)

In [ ]:
# Fetch financial statements for a single company and save to DB
def fetch_and_store_10k_data(ticker):
    stock = yf.Ticker(ticker)

    # Fetch and convert financial statements
    financials_json = convert_columns_to_year(stock.financials)
    balance_sheet_json = convert_columns_to_year(stock.balance_sheet)
    cashflow_json = convert_columns_to_year(stock.cashflow)

    # Store each type into the database
    save_financials_to_db(ticker, "financials", financials_json)
    save_financials_to_db(ticker, "balance_sheet", balance_sheet_json)
    save_financials_to_db(ticker, "cashflow", cashflow_json)

**4. Fetch 10-K reports for NASDAQ-100 companies using yfinance and store them in the database.**

In [ ]:
# Test with a single company (Apple)
fetch_and_store_10k_data("AAPL")

In [ ]:
# List of NASDAQ-100 tickers
nasdaq_100_tickers = ['AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AMAT', 'AMD', 'AMGN',
 'AMZN', 'ANSS', 'APP', 'ARM', 'ASML', 'AVGO', 'AXON', 'AZN', 'BIIB', 'BKNG',
 'BKR', 'CCEP', 'CDNS', 'CDW', 'CEG', 'CHTR', 'CMCSA', 'COST', 'CPRT', 'CRWD',
 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTSH', 'DASH', 'DDOG', 'DXCM', 'EA', 'EXC',
 'FANG', 'FAST', 'FTNT', 'GEHC', 'GFS', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX',
 'INTC', 'INTU', 'ISRG', 'KDP', 'KHC', 'KLAC', 'LIN', 'LRCX', 'LULU', 'MAR',
 'MCHP', 'MDB', 'MDLZ', 'MELI', 'META', 'MNST', 'MRVL', 'MSFT', 'MSTR', 'MU',
 'NFLX', 'NVDA', 'NXPI', 'ODFL', 'ON', 'ORLY', 'PANW', 'PAYX', 'PCAR', 'PDD',
 'PEP', 'PLTR', 'PYPL', 'QCOM', 'REGN', 'ROP', 'ROST', 'SBUX', 'SNPS', 'TEAM',
 'TMUS', 'TSLA', 'TTD', 'TTWO', 'TXN', 'VRSK', 'VRTX', 'WBD', 'WDAY', 'XEL', 'ZS']

In [ ]:
# Loop through all NASDAQ-100 companies and store their 10-K data
for ticker in nasdaq_100_tickers:
    fetch_and_store_10k_data(ticker)

print("✅ 10-K financial data from 2020–2025 for NASDAQ-100 companies has been stored in SQLite!")

✅ 2020-2025 NASDAQ-100 公司的 10-K 數據已存入 SQL！
